In [1]:
import pandas as pd
import numpy as np

### process different phenotype files

In [2]:
polypred_max1 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max1.prs",sep='\t')
polypred_max3 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max3.prs",sep='\t')
polypred_max5 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max5.prs",sep='\t')
polypred_max7 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max7.prs",sep='\t')
polypred_max10 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max10.prs",sep='\t')
manifest = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/SampleManifest_DS_2021.02.19_ALL.txt", sep='\t') ## the one with mapping info
phenotype1 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADNIPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')
phenotype2 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSPCaseControlPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')
phenotype3 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSPFamilyBasedPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')
phenotype4 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/PSPCBDPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')

mapping = pd.read_excel("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSP-FUS1-Study-to-SampleID-mapping.xlsx")
ancestry = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSP_ethnicity_prediction.tsv", sep = "\t")
phenotype = pd.read_csv("/gpfs/commons/home/tlin/data/phenotype_0219.2021.tsv", sep = '\t')


## check ancestry file.

In [10]:
manifest.columns


Index(['SUBJID', 'SampleID', 'BODY_SITE', 'ANALYTE_TYPE', 'Sequencing_Center',
       'Sequencing_Platform', 'SAMPLE_USE', 'Technical_Replicate', 'Study_DSS',
       'Sample_Set'],
      dtype='object')

In [3]:
ancestry[ancestry.sample_id.isin(manifest.SampleID)].shape  ## all ADSP data have SUBJID to mapped to. 

(16906, 18)

In [4]:
## add SUBJID to ancestry
ancestry = ancestry.merge(manifest, left_on="sample_id", right_on="SampleID")
ancestry.shape  ## (16906, 28)

(16906, 28)

In [5]:
ancestry[ancestry.SUBJID.isin(phenotype1.SUBJID)].shape   #1566
ancestry[ancestry.SUBJID.isin(phenotype2.SUBJID)].shape  # 12426
ancestry[ancestry.SUBJID.isin(phenotype3.SUBJID)].shape  #964

## SUM=14956

(964, 28)

In [19]:
1566+12426+964

14956

In [6]:
ancestry[(ancestry.SUBJID.isin(phenotype1.SUBJID)) | (ancestry.SUBJID.isin(phenotype2.SUBJID)) | (ancestry.SUBJID.isin(phenotype3.SUBJID))].shape

(14914, 28)

In [7]:
ancestry[(ancestry.SUBJID.isin(phenotype2.SUBJID)) & (ancestry.SUBJID.isin(phenotype3.SUBJID))].shape  ## 42 duplicated found in ADSP case control/ Family based ## 14956-14914 = 42

(42, 28)

In [12]:
ancestry[(ancestry.SUBJID.isin(phenotype2.SUBJID)) & (ancestry.SUBJID.isin(phenotype3.SUBJID))].drop_duplicates().shape

(42, 28)

In [14]:
ancestry[ancestry.duplicated()]

,sample_id,population_pred,max_probability,prob_0,prob_1,prob_2,prob_3,prob_4,distance,threshold,...,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set


In [41]:
16906-14914

1992

In [15]:
ancestry[(~ancestry.SUBJID.isin(phenotype1.SUBJID)) & (~ancestry.SUBJID.isin(phenotype2.SUBJID)) & (~ancestry.SUBJID.isin(phenotype3.SUBJID))].shape

(1992, 28)

In [16]:
ancestry[ancestry.sample_id.isin(polypred_max1.IID)].shape

(16906, 28)

In [17]:
ancestry[ancestry.SUBJID.isin(phenotype.SUBJID)].shape

(11903, 28)

In [24]:
phenotype2 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSPCaseControlPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')
phenotype2.shape

(27547, 27)

In [35]:
phenotype2.dropna(subset = ["PrevAD",'IncAD'],how="all").shape

(26163, 27)

In [34]:
phenotype2.dropna(subset = ["PrevAD",'IncAD']).shape

(23479, 27)

### backward merge with Phenotype

In [36]:
## there is no NAs in phenotype1
phenotype1["Diagnosis"] = phenotype1[['PrevAD', 'IncAD']].max(axis=1).astype(int)
phenotype1['Age'] = np.where(phenotype1.Diagnosis == 1,phenotype1.Age_AD_onset, phenotype1.Age_current)

phenotype2[["PrevAD",'IncAD']] = phenotype2[["PrevAD",'IncAD']].fillna(value = -1)
phenotype2["Diagnosis"] = phenotype2[['PrevAD', 'IncAD']].max(axis=1).astype(int)


phenotype3['Diagnosis'] = 0
phenotype3.loc[:,'Diagnosis'][(phenotype3.AD<= 3) & (phenotype3.AD > 0)] =1

/nfs/sw/jupyter/jupyter3-4.1/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [42]:
wanted_col = ["SUBJID","Sex","Age","APOE","Race","Diagnosis"]

def check_na(df,name):
    print("phenotye: ",name)
    print("data size: ", df.loc[:,wanted_col].shape[0])
    print("NAs in Diagnosis: ", df[df.Diagnosis == -1].shape[0])
    print("NAs in Age/APOE/Race: ", df.loc[:,wanted_col].shape[0] -df.loc[:,wanted_col].dropna(subset = ["Age","APOE","Race"]).shape[0],"\n")
   

    
check_na(phenotype1,"ADNI Phenotypes")
check_na(phenotype2,"ADSP CaseControlPhenotypes")
check_na(phenotype3,"ADSP FamilyBasedPhenotypes")



phenotye:  ADNI Phenotypes
data size:  1566
NAs in Diagnosis:  0
NAs in Age/APOE/Race:  9 

phenotye:  ADSP CaseControlPhenotypes
data size:  27547
NAs in Diagnosis:  1384
NAs in Age/APOE/Race:  2161 

phenotye:  ADSP FamilyBasedPhenotypes
data size:  12424
NAs in Diagnosis:  0
NAs in Age/APOE/Race:  8469 



In [74]:
phenotype_merge2 = pd.concat([phenotype1.loc[:,wanted_col],phenotype2.loc[:,wanted_col],phenotype3.loc[:,wanted_col]])

print(phenotype.shape[0])
print(phenotype_merge2.shape[0])
print(phenotype_merge2[phenotype_merge2.Diagnosis == -1].shape[0])



30341
41537
1384


In [75]:
phenotype_merge2 = phenotype_merge2.sort_values("SUBJID")
phenotype_merge2.reset_index(drop=True, inplace=True)

In [80]:
phenotype_merge2.loc[phenotype_merge2["Age"] == "90+","Age"] = 90

In [76]:
phenotype_merge2.shape

(41537, 6)

In [81]:
phenotype_merge2 = phenotype_merge2.drop_duplicates()

In [82]:
duplicated = phenotype_merge2[phenotype_merge2.SUBJID.duplicated()].SUBJID
duplicated.shape

(36,)

In [83]:
phenotype_merge2[phenotype_merge2.SUBJID.isin(duplicated)]

,SUBJID,Sex,Age,APOE,Race,Diagnosis
16996,A-LOAD-LD000600,0,72,33.0,5.0,1
16997,A-LOAD-LD000600,0,68,33.0,5.0,1
17130,A-LOAD-LD000909,0,65,34.0,5.0,1
17131,A-LOAD-LD000909,0,70,34.0,5.0,1
17347,A-LOAD-LD001254,0,79,34.0,5.0,1
17348,A-LOAD-LD001254,0,82,34.0,5.0,1
17690,A-LOAD-LD001837,1,76,34.0,5.0,1
17691,A-LOAD-LD001837,1,80,34.0,5.0,1
17857,A-LOAD-LD002391,1,90,33.0,5.0,1
17858,A-LOAD-LD002391,1,88,33.0,5.0,1


In [121]:
ancestry[ancestry.SUBJID.isin(phenotype2.SUBJID)].shape  

(14914, 28)

In [213]:
ancestry.shape

(16906, 28)

In [225]:
ancestry = ancestry.merge(phenotype_merge2, on = "SUBJID")

In [227]:
ancestry.shape

(14956, 33)

In [172]:
ancestry.merge(phenotype_merge2, on = "SUBJID")[ancestry.merge(phenotype_merge2, on = "SUBJID").Diagnosis == -1].shape ## in 14914 samples, there are 741 without AD lables

(741, 33)

In [182]:
print(ancestry[ancestry.SUBJID.isin(phenotype.SUBJID)].shape)
print(ancestry[ancestry.SUBJID.isin(phenotype_merge2.SUBJID)].shape)
print(ancestry[ancestry.SUBJID.isin(phenotype_merge2.SUBJID) & ~ancestry.SUBJID.isin(phenotype.SUBJID)].shape)
print(ancestry.sample_id.drop_duplicates().shape)

(11903, 28)
(14914, 28)
(3011, 28)
(16906,)


In [43]:
polypred_max1[polypred_max1.IID.isin(ancestry.sample_id)].shape # 16906
polypred_max1[polypred_max1.IID.isin(phenotype_merge2.)]

SyntaxError: invalid syntax (<ipython-input-43-ac300bfcbe15>, line 2)

In [223]:
ancestry.shape

(16906, 28)

In [224]:
phenotype_merge2.shape

(41537, 6)

In [190]:
missing_in_merging = ancestry[ancestry.SUBJID.isin(phenotype_merge2.SUBJID) & ~ancestry.SUBJID.isin(phenotype.SUBJID)].SUBJID  #3011

In [210]:
phenotype_merge2[(phenotype_merge2.SUBJID.isin(missing_in_merging)) & (phenotype_merge2.Diagnosis != -1)] ##2245

,SUBJID,Sex,Age,APOE,Race,Diagnosis
1406,A-ADC-AD000033,0,65,44.0,5.0,1
1410,A-ADC-AD000039,0,59,34.0,5.0,1
1442,A-ADC-AD000103,0,65,44.0,5.0,1
1443,A-ADC-AD000104,0,64,34.0,5.0,1
1464,A-ADC-AD000142,0,64,44.0,5.0,1
1479,A-ADC-AD000163,0,65,44.0,5.0,1
1480,A-ADC-AD000165,0,63,44.0,5.0,1
1493,A-ADC-AD000203,0,54,34.0,5.0,1
1584,A-ADC-AD000300,0,61,34.0,5.0,1
1602,A-ADC-AD000332,1,65,44.0,5.0,1


In [205]:
phenotype3[phenotype3.SUBJID=="A-LOAD-LD014234"]

,SUBJID,FamID,Father,Mother,Sex,AD,Age,Age_baseline,APOE,AUTOPSY,...,Base_AD,Base_Age,Base_AUTOPSY,Base_Braak,Base_Version,Update_baseline,Update_latest,Update_ADstatus,Correction,Diagnosis
12423,A-LOAD-LD014234,LD1517F,0,0,1,9,NaN,NaN,NaN,NaN,...,9,NaN,NaN,NaN,ng00067.v5,0,1,0,0,0


In [206]:
phenotype_merge2[phenotype_merge2.SUBJID=="A-LOAD-LD014234"]

,SUBJID,Sex,Age,APOE,Race,Diagnosis
12423,A-LOAD-LD014234,1,NaN,NaN,NaN,0


In [207]:
phenotype[phenotype.SUBJID=="A-LOAD-LD014234"]

,SUBJID,Diagnosis,Sex,Age,APOE,Race,SampleID


### check conflict & make sure SUBJID are all unique


In [186]:
## There is no conflict
## remove duplicated rows
phenotype = phenotype.drop_duplicates() ## 81 duplicated rows
duplicates = phenotype[phenotype.SUBJID.duplicated()].SUBJID
duplicates.shape ## 32 DUPLICATEED
phenotype.reset_index(drop=True, inplace=True)
phenotype.shape ##(37388,6)

(37388, 6)

In [187]:
## deal with duplicated SUBJID
phenotype.loc[phenotype["Age"] == "90+","Age"] = 90
df = phenotype[phenotype.SUBJID.isin(duplicates)].sort_values("SUBJID")
df.reset_index(drop=True, inplace=True)
df.Age = df.Age.astype(int)

## we took the younger of the two 
### note: df only contain AD patients
for i in range(0,63,2):
    if (df["Diagnosis"][i] == 1):
        #print(df['Age'][i],df['Age'][i+1])
        if (df['Age'][i] > (df['Age'][i+1])):
            df = df.drop(i, axis=0)
        else:
            df = df.drop(i+1, axis=0)


In [188]:
## making sure every SUBJID is unique 
phenotype = phenotype[~phenotype.SUBJID.isin(duplicates)]
print(phenotype.shape)
phenotype = pd.concat([phenotype,df])
print(phenotype.shape)
phenotype.reset_index(drop=True, inplace=True)

(37324, 6)
(37356, 6)


### add sample ID to phenotype file

In [170]:
print(manifest.head())

           SUBJID                     SampleID BODY_SITE ANALYTE_TYPE  \
0  A-ACT-AC000004  A-ACT-AC000004-BL-UPN-15872     Blood          DNA   
1  A-ACT-AC000007   A-ACT-AC000007-BL-UPN-6888     Blood          DNA   
2  A-ACT-AC000008   A-ACT-AC000008-BL-UPN-8307     Blood          DNA   
3  A-ACT-AC000010  A-ACT-AC000010-BL-UWA-15286     Blood          DNA   
4  A-ACT-AC000014  A-ACT-AC000014-BL-UPN-23967     Blood          DNA   

  Sequencing_Center  Sequencing_Platform SAMPLE_USE  Technical_Replicate  \
0             WashU  Illumina HiSeq 2000        WES                    0   
1             WashU  Illumina HiSeq 2000        WES                    0   
2             WashU  Illumina HiSeq 2000        WES                    0   
3             WashU  Illumina HiSeq 2000        WES                    0   
4             WashU  Illumina HiSeq 2000        WES                    0   

  Study_DSS Sample_Set  
0  sa000001   snd10000  
1  sa000001   snd10000  
2  sa000001   snd10000  
3  s

In [192]:
phenotype = phenotype.merge(manifest.loc[:,["SUBJID","SampleID"]], on= "SUBJID")

(30341, 7)

### combine polypred result with phenotypic files

In [194]:
polypred_max1 = polypred_max1.merge(phenotype, left_on = "IID", right_on = "SampleID").drop(["IID"],axis=1)
polypred_max1 = polypred_max1.loc[:,["SUBJID","SampleID","FID","PRS","Diagnosis","Sex","Age","APOE","Race"]]

In [6]:
def combine_phenotype_polypred(polypred, phenotype):
    polypred = polypred.merge(phenotype, left_on = "IID", right_on = "SampleID").drop(["IID"],axis=1)
    polypred = polypred.loc[:,["SUBJID","SampleID","FID","PRS","Diagnosis","Sex","Age","APOE","Race"]]
    return(polypred)

In [7]:
polypred_pheno_max1 = combine_phenotype_polypred(polypred_max1, phenotype)
polypred_pheno_max3 = combine_phenotype_polypred(polypred_max3, phenotype)
polypred_pheno_max5 = combine_phenotype_polypred(polypred_max5, phenotype)
polypred_pheno_max7 = combine_phenotype_polypred(polypred_max7, phenotype)
polypred_pheno_max10 = combine_phenotype_polypred(polypred_max10, phenotype)

In [8]:
print("check match/unmatch:")
print("max_SNP = 1, before merging:%s, after merging:%s"%(polypred_max1.shape[0],polypred_pheno_max1.shape[0]) )
print("max_SNP = 3, before merging:%s, after merging:%s"%(polypred_max3.shape[0],polypred_pheno_max3.shape[0]) )
print("max_SNP = 5, before merging:%s, after merging:%s"%(polypred_max5.shape[0],polypred_pheno_max5.shape[0]) )
print("max_SNP = 7, before merging:%s, after merging:%s"%(polypred_max7.shape[0],polypred_pheno_max7.shape[0]) )
print("max_SNP = 10, before merging:%s, after merging:%s"%(polypred_max10.shape[0],polypred_pheno_max10.shape[0]) )


check match/unmatch:
max_SNP = 1, before merging:16906, after merging:11903
max_SNP = 3, before merging:16906, after merging:11903
max_SNP = 5, before merging:16906, after merging:11903
max_SNP = 7, before merging:16906, after merging:11903
max_SNP = 10, before merging:16906, after merging:11903


In [213]:
polypred_pheno_max1.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_1.tsv",index=False, sep = '\t')
polypred_pheno_max3.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_3.tsv",index=False, sep = '\t')
polypred_pheno_max5.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_5.tsv",index=False, sep = '\t')
polypred_pheno_max7.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_7.tsv",index=False, sep = '\t')
polypred_pheno_max10.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp10.tsv",index=False, sep = '\t') 

In [229]:
print(polypred_pheno_max1.dropna().shape)
print(polypred_pheno_max1.shape)

(11555, 9)
(11903, 9)


In [231]:
polypred_pheno_max1.isna().any()

SUBJID       False
SampleID     False
FID          False
PRS          False
Diagnosis    False
Sex          False
Age           True
APOE          True
Race          True
dtype: bool

In [226]:
polypred_pheno_max1[polypred_pheno_max1.isna().any(axis =1)]

,SUBJID,SampleID,FID,PRS,Diagnosis,Sex,Age,APOE,Race
65,A-CUHS-CU000259,A-CUHS-CU000259-BL-COL-57383BL1,A-CUHS-CU000259-BL-COL-57383BL1,-0.00160,0,0,NaN,33.0,5.0
70,A-CUHS-CU000650,A-CUHS-CU000650-BL-COL-38425BL1,A-CUHS-CU000650-BL-COL-38425BL1,-0.00117,0,0,NaN,33.0,6.0
83,A-CUHS-CU001264,A-CUHS-CU001264-BL-COL-34461BL2,A-CUHS-CU001264-BL-COL-34461BL2,-0.00103,0,1,NaN,34.0,6.0
90,A-CUHS-CU001475,A-CUHS-CU001475-BL-COL-41299BL1,A-CUHS-CU001475-BL-COL-41299BL1,-0.00100,0,1,NaN,33.0,6.0
115,A-CUHS-CU002844,A-CUHS-CU002844-BL-COL-49664BL1,A-CUHS-CU002844-BL-COL-49664BL1,-0.00090,0,0,NaN,34.0,6.0
140,A-MIA-UM001159,A-MIA-UM001159-BL-MIA-201104519,A-MIA-UM001159-BL-MIA-201104519,-0.00103,0,1,55,33.0,NaN
169,A-LOAD-LD005871,A-LOAD-LD005871-BL-NCR-04AD6144,A-LOAD-LD005871-BL-NCR-04AD6144,-0.00164,0,0,NaN,34.0,5.0
206,A-CUHS-CU000444,A-CUHS-CU000444-BL-COL-32023BL2,A-CUHS-CU000444-BL-COL-32023BL2,-0.00151,0,1,NaN,33.0,5.0
281,A-CUHS-CU001234,A-CUHS-CU001234-BL-COL-41585BL1,A-CUHS-CU001234-BL-COL-41585BL1,-0.00125,0,0,NaN,33.0,6.0
304,A-CUHS-CU001594,A-CUHS-CU001594-BL-COL-46336BL1,A-CUHS-CU001594-BL-COL-46336BL1,0.00037,0,1,NaN,33.0,5.0


In [227]:
#phenotype.to_csv("/gpfs/commons/home/tlin/data/phenotype_0219.2021.tsv", sep = '\t', index = False)

### find the sample ID of those polypred data that have no phenotypic info


In [26]:
type(polypred_max1)

pandas.core.frame.DataFrame

In [12]:
polypred_pheno_max1.columns

Index(['SUBJID', 'SampleID', 'FID', 'PRS', 'Diagnosis', 'Sex', 'Age', 'APOE',
       'Race'],
      dtype='object')

In [36]:
def find_missing(polypred, polypred_pheno):
    missing = polypred[~polypred.IID.isin(polypred_pheno.SampleID)]
    return(missing)



In [44]:
find_missing(polypred_max1, polypred_pheno_max1).to_csv("/gpfs/commons/home/tlin/data/missingID_max1.tsv", sep = '\t')
find_missing(polypred_max3, polypred_pheno_max3).to_csv("/gpfs/commons/home/tlin/data/missingID_max3.tsv", sep = '\t')
find_missing(polypred_max5, polypred_pheno_max5).to_csv("/gpfs/commons/home/tlin/data/missingID_max5.tsv", sep = '\t')
find_missing(polypred_max7, polypred_pheno_max7).to_csv("/gpfs/commons/home/tlin/data/missingID_max7.tsv", sep = '\t')
find_missing(polypred_max10, polypred_pheno_max10).to_csv("/gpfs/commons/home/tlin/data/missingID_max10.tsv", sep = '\t')

In [15]:
16906-11903

5003

In [40]:
find_missing(polypred_max1, polypred_pheno_max1)

,FID,IID,PRS
951,A-ACT-AC002132-BL-NCR-12AD50655,A-ACT-AC002132-BL-NCR-12AD50655,-0.00132
1031,A-ADC-AD000882-BL-NCR-09AD14878,A-ADC-AD000882-BL-NCR-09AD14878,-0.00171
1038,A-ADC-AD000922-BL-NCR-09AD14947,A-ADC-AD000922-BL-NCR-09AD14947,-0.00102
1043,A-ADC-AD000954-BL-NCR-09AD16956,A-ADC-AD000954-BL-NCR-09AD16956,-0.00097
1044,A-ADC-AD000955-BL-NCR-09AD17029,A-ADC-AD000955-BL-NCR-09AD17029,-0.00145
1050,A-ADC-AD001008-BL-NCR-09AD16921,A-ADC-AD001008-BL-NCR-09AD16921,-0.00071
1051,A-ADC-AD001015-BL-NCR-09AD14954,A-ADC-AD001015-BL-NCR-09AD14954,-0.00178
1055,A-ADC-AD001042-BL-NCR-09AD14998,A-ADC-AD001042-BL-NCR-09AD14998,-0.00084
1056,A-ADC-AD001044-BL-NCR-09AD14988,A-ADC-AD001044-BL-NCR-09AD14988,-0.00045
1069,A-ADC-AD001129-BL-NCR-09AD14028,A-ADC-AD001129-BL-NCR-09AD14028,-0.00204


In [ ]:
A-ACT-AC002132-BL-NCR-12AD50655

In [50]:
phenotype[phenotype.SampleID == 'A-ACT-AC002132-BL-NCR-12AD50655']


,SUBJID,Diagnosis,Sex,Age,APOE,Race,SampleID


,SUBJID,Sex,Diagnosis,AgeOnset,AgeDeath,Race,Duplicate_SUBJID,Comments,Latest_Update_Version,Base_Diagnosis,Base_Version,Update_Baseline,Update_Latest,Update_Diagnosis,Correction
0,P-ABN-CBD6614,1,2,67,73,5.0,0,NaN,ng00067.v2,2,ng00067.v2,0,0,0,0
1,P-ABN-PSP5079,1,1,71,76,NaN,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
2,P-ABN-PSP5080,1,1,68,73,NaN,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
3,P-ABN-PSP5081,0,1,72,75,NaN,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
4,P-ABN-PSP5082,1,1,68,73,NaN,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
5,P-ABN-PSP5083,1,1,74,80,NaN,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
6,P-ABN-PSP5084,0,1,68,73,NaN,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
7,P-ABN-PSP5085,0,1,75,79,NaN,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
8,P-ABN-PSP5086,0,1,63,73,5.0,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
9,P-ABN-PSP5087,0,1,57,59,5.0,NC,NaN,ng00067.v5,1,ng00067.v5,0,1,0,0
